In [ ]:
using Plots
using PAM
using FUSE

In [ ]:
import IMAS;
# start with ods for DIIID case
dd = IMAS.json2imas("/Users/galinaavdeeva/.julia/dev/PAM/inputs/ods_d3d.json");
eqt = dd.equilibrium.time_slice[]
IMAS.flux_surfaces(eqt, IMAS.first_wall(dd.wall)...);
# add missing (in OMFIT) time_slice to ods
dd_pellet = IMAS.json2imas("/Users/galinaavdeeva/.julia/dev/PAM/examples/ods.txt");
dd.core_profiles.profiles_1d[1].time = dd.equilibrium.time_slice[1].time;
# save dd pellet structure for future use
old_pellets = deepcopy(dd_pellet.pellets);


In [ ]:
# add and populate 'layer' which are needed to PAM

empty!(dd.pellets)
resize!(dd.pellets.time_slice)
resize!(dd.pellets.time_slice[].pellet, 1)
resize!(dd.pellets.time_slice[].pellet[1].layer, 1)
resize!(dd.pellets.time_slice[].pellet[1].layer[1].species, 1);
#resize!(dd.pellets.time_slice[].pellet[1].layer[1].species, 2);


In [ ]:
# modify and populate other nessesary entries
dd.pellets.time_slice[].pellet[1].layer[1].thickness = 0.001



dd.pellets.time_slice[].pellet[1].shape = old_pellets.time_slice[].pellet[1].shape
dd.pellets.time_slice[].pellet[1].path_geometry = old_pellets.time_slice[].pellet[1].path_geometry
dd.pellets.time_slice[].pellet[1].path_profiles = old_pellets.time_slice[].pellet[1].path_profiles
dd.pellets.time_slice[].pellet[1].velocity_initial = 120.0 #old_pellets.time_slice[].pellet[1].velocity_initial
props = IMAS.ion_properties(:C)

dd.pellets.time_slice[].pellet[1].layer[1].species[1].z_n = props.z_n
dd.pellets.time_slice[].pellet[1].layer[1].species[1].a = props.a
dd.pellets.time_slice[].pellet[1].layer[1].species[1].label = props.label


dd.pellets.time_slice[].pellet[1].layer[1].species[1].fraction = 1.0


dd.pellets.time_slice[].pellet[1].path_geometry.second_point.z = 0.0
dd.pellets.time_slice[].pellet[1].path_geometry.second_point.phi = 0.0
dd.pellets.time_slice[].pellet[1].path_geometry.second_point.r = 1.9
dd.pellets.time_slice[].pellet[1].path_geometry.first_point.r = 2.3




radii = cumsum([layer.thickness for layer in dd.pellets.time_slice[].pellet[1].layer])
dd.pellets.time_slice[].pellet[1].shape.size[1] = 0.001;#radii[end]




In [ ]:
# run PAM

inputs=(
   t0 = 0,
   tf = 0.0016,
   dt = 0.0001, #  time step of simulations
   drift_model= :Parks,
   BtDependance=true,
)



pellet = PAM.run_PAM(dd, inputs);
#plot(pellet.time, pellet.ablation_rate;marker=:circle, markersize=1)
#display(plot!())

#plot(dd.equilibrium;cx=true)
#plot!(pellet)
plot(pellet.R_drift,markershape=:circle)
#plot(pellet.ρ, pellet.radius,markershape=:circle)

In [ ]:
time= [0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0, 1.1, 1.2000000000000002, 1.3, 1.4000000000000001] ;
rp= [0.1, 0.1, 0.1, 0.09999414910178023, 0.09996872578498792, 0.09993572421237616, 0.09989676602572373, 0.09985046637706875, 0.0997953212457394, 0.09972981598708504, 0.09965240925575566, 0.09956153544611122, 0.09945562219267796, 0.09933309558454219, 0.09919237845260413] ;

In [ ]:
shift_r= [0.0, 0.0, 0.0, 0.019802188335738435, 0.028670636408833605, 0.033005884414915254, 0.0381148342417621, 0.04405238670726286, 0.05071433050632024, 0.05819507366730825, 0.06598015689173309, 0.07471327911208014, 0.08343407643992728, 0.09139138047276951, 0.0] ;

In [ ]:
plot(time,shift_r)
plot!(pellet.time*1e+3, pellet.R_drift)

In [ ]:

plot(abc,markershape=:circle)
plot!(pellet.radius*1e2,markershape=:circle)

In [ ]:
plot(dd.equilibrium; cx=true)
plot!(pellet)

In [ ]:
plot(pellet.ablation_rate)

In [ ]:
pypelr=[0.1,        0.1,        0.1,        0.09990515, 0.0981528,  0.0948701,
 0.09087481, 0.08609535, 0.08029067, 0.07313583, 0.0641541,  0.05252969,
 0.03642787, 0.00484595, 0.        ]
pytime=[0.,  0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8 , 0.9 , 1.,  1.1, 1.2, 1.3, 1.4]
plot(pellet.time*1e+3, pellet.radius,markershape=:circle)
plot!(pytime, pypelr*1e-2,markershape=:circle)



In [ ]:
R1 = range(1.7, 2.5,100)
Z1 = R1 .* 0.0

# rho interpolant
Rgrid, Zgrid, RHO_interpolant = IMAS.ρ_interpolant(eqt)

# check the pellet position
rho_start = RHO_interpolant.(R1, Z1);



In [ ]:
print(pellet.R_drift)

In [ ]:
pyRd = [0.,         0.,         0.,         0.02979549, 0.10370448, 0.12239645,
 0.13416608, 0.14645173, 0.15804974, 0.16694494, 0.1684416,  0.16225686,
 0.13513904, 0.02869696, 0.        ];

In [ ]:
plot(pellet.R_drift)
plot!(pyRd)

In [ ]:
scatter(R1, rho_start)
plot!(eqt.profiles_1d.r_outboard,eqt.profiles_1d.rho_tor_norm)
vline!([maximum(Rgrid)])

In [ ]:
# contour(sqrt.(abs.(eqt.profiles_2d[1].phi)), levels=1000)
# for item in [:b_field_r, :b_field_tor, :b_field_z, :phi]
#     if IMAS.hasdata(eqt.profiles_2d[1], item)
#         empty!(eqt.profiles_2d[1], item)
#     end
# end
IMAS.flux_surfaces(eqt, IMAS.first_wall(dd.wall)...)
eqt.profiles_2d[1]

In [ ]:
contour(sqrt.(abs.(eqt.profiles_2d[1].phi)), levels=1000)

In [ ]:
plot(pellet.radius)

In [ ]:
plot(pellet.density_source[450,:])


In [ ]:
print(pellet.ablation_rate*6.022e23*0.5/0.5)

In [ ]:
0.5*(3.016+2.014)

In [ ]:
39*(0.5^(5/3))*((1/0.2)^(4/3))/4/π

In [ ]:
print(Na)

In [ ]:
a=m_p
print(a)

In [ ]:
using PhysicalConstants.CODATA2018

In [ ]:
import PhysicalConstants.CODATA2014: c_0, ε_0, μ_0, m_p

In [ ]:
sqrt(2*2)

In [ ]:
dd.equilibrium

In [ ]:
import IMAS;
# start with ods for DIIID case
dd2 = IMAS.json2imas("/Users/galinaavdeeva/.julia/dev/PAM/inputs/ods_d3d.json");
eqt = dd2.equilibrium.time_slice[]
IMAS.flux_surfaces(eqt, IMAS.first_wall(dd.wall)...);
surfaces = IMAS.trace_surfaces(eqt, IMAS.first_wall(dd.wall)...)

In [ ]:
plot!(dd2.equilibrium)